In [ ]:
import logging
import warnings
logging.getLogger('fbprophet').setLevel(logging.ERROR)
logging.getLogger('statsmodels').setLevel(logging.WARNING)
logging.getLogger('arch').setLevel(logging.WARNING)
logging.getLogger('croston').setLevel(logging.WARNING)
logging.getLogger('pandas').setLevel(logging.WARNING)
logging.getLogger('pmdarima').setLevel(logging.WARNING)
logging.getLogger('causalimpact').setLevel(logging.WARNING)
warnings.filterwarnings("ignore", category=RuntimeWarning)

: 

In [ ]:
import forecast_engine.modelling.utils.logger as logger
import forecast_engine.modelling.thunderbird as thunderbird

In [ ]:
thunderbird.thunderbird.instantiate_inputs()
thunderbird.thunderbird.preprocess_inputs()

In [ ]:
thunderbird.thunderbird.thunder_fuel()

In [ ]:
thunderbird.thunderbird.switch_on()

PREPROCESS

In [110]:
import forecast_engine.preprocess.nodes as preprocess

SALES ORDER

In [111]:
path = r""
sales_data, df = preprocess.read_prep_sales_order(path)
dates, act_dates = preprocess.prep_dates_list(df)

In [112]:
sub_season = [str(x)+'_Season' for x in dates]
df[sub_season] = df.apply( lambda x : preprocess.decomposition(x,dates,df.columns), axis = 1, result_type="expand")

In [113]:
df['intermittence'] = df.apply(lambda x : preprocess.intermittence(x), axis =1)
df['cov'] = df[dates].apply(lambda x : preprocess.cov(x), axis =1)
df['seq - flag'] = df.apply(lambda x : preprocess.seq_flag(x,dates,df.columns) , axis = 1)
df.drop(['intermittence','cov'],axis =1,inplace=True)

In [114]:
df.drop(['intermittence','cov'],axis =1,inplace=True)

WEATHER DATA

In [115]:
path = r"Weather Data/Weather Data.xlsx"
wthr_data = preprocess.read_prep_weather(path)

In [116]:
wthr_data

,Country,2019 - 01_Avg Temp (C),2019 - 02_Avg Temp (C),2019 - 03_Avg Temp (C),2019 - 04_Avg Temp (C),2019 - 05_Avg Temp (C),2019 - 06_Avg Temp (C),2019 - 07_Avg Temp (C),2019 - 08_Avg Temp (C),2019 - 09_Avg Temp (C),...,2021 - 10_Relative Humidity (%),2021 - 11_Relative Humidity (%),2021 - 12_Relative Humidity (%),2022 - 01_Relative Humidity (%),2022 - 02_Relative Humidity (%),2022 - 03_Relative Humidity (%),2022 - 04_Relative Humidity (%),2022 - 05_Relative Humidity (%),2022 - 06_Relative Humidity (%),2022 - 07_Relative Humidity (%)
0,Mexico,15.100685,17.501682,18.93809,20.299129,22.072156,21.325376,20.17738,20.719087,19.970685,...,74.688081,67.689772,61.966188,63.661877,58.495351,49.211327,53.776839,52.436179,68.25951,72.98732


MACROECONOMIC DATA

In [117]:
path = r"/Macro-economics"
macro = preprocess.read_macro_files(path, country = ['Mexico'])

In [118]:
macro

,2019 - 01_Inflation Data,2019 - 02_Inflation Data,2019 - 03_Inflation Data,2019 - 04_Inflation Data,2019 - 05_Inflation Data,2019 - 06_Inflation Data,2019 - 07_Inflation Data,2019 - 08_Inflation Data,2019 - 09_Inflation Data,2019 - 10_Inflation Data,...,2022 - 04_Lending Rate,2022 - 05_Lending Rate,2022 - 06_Lending Rate,2022 - 07_Lending Rate,2022 - 08_Lending Rate,2022 - 09_Lending Rate,2022 - 10_Lending Rate,2022 - 11_Lending Rate,2022 - 12_Lending Rate,Country
0,0.0437,0.0394,0.04,0.0441,0.0428,0.0395,0.0378,0.0316,0.03,0.0302,...,0.065,0.07,0.0775,0.0775,0.085,0.0925,0.0925,0.1,0.105,Mexico


MERGE DATA

In [119]:
df['Country'] = 'Mexico'
data = pd.merge(df, macro, on = 'Country', how = 'left')
data = pd.merge(data, wthr_data, on = 'Country', how = 'left')
data.drop('Country',axis=1,inplace=True)
data.rename(columns={'Material':'index'},inplace=True)
data.columns = [str(i) for i in list(data.columns) ]
    

Forecast Engine

In [144]:
#Flags
debug_flag = 0
debug_models = []
outlier_flag = 1
select_variable_flag = 0
all_at_once_flag = 1
months_to_forecast = 3
max_validation = 4
validation_window = 2
single_series = 1
multivariate_flag = 1

univariate_models = {'hw_ets_r_MNN':holt_winters.hw_ets_r_MNN, 'hw_ets_ATA':holt_winters.hw_ets_ATA, 'hw_ets_AFA':holt_winters.hw_ets_AFA, 
                    'hw_ets_ATN':holt_winters.hw_ets_ATN, 'hw_ets_NFA':holt_winters.hw_ets_NFA, 'hw_ets_NFN':holt_winters.hw_ets_NFN, 'hw_ets_AFN':holt_winters.hw_ets_AFN,
                    'hw_ets_r_MAA':holt_winters.hw_ets_r_MAA,'hw_ets_r_MAN':holt_winters.hw_ets_r_MAN, 'hw_ets_r_MNA':holt_winters.hw_ets_r_MNA,
                    
                    'mean_model':naive_models.mean_model, 'median_model':naive_models.median_model, 'naive':naive_models.naive, 'naive_ets':naive_models.naive_ets,
                    'snaive':naive_models.snaive, 'snaive_twoseasons':naive_models.snaive_twoseasons,
                    
                    'rw_drift':rw_drift.rw_drift, 'croston':croston.croston, 'arima':arima.arima, 'garch':garch.garch, 'linearregression_mv':linear_regression_mv.linearregression_mv,

                    'xgboost_mv':xgboost_mv.xgboost_mv,'prophet':prophet.prophet,'causalimpact':causal_impact.causalimpact,'light_gbm':light_gbm.lgbm,
                    'random_forest_mv' : random_forest_mv.random_forest_mv}


multivariate_models = {'linearregression_mv':linear_regression_mv.linearregression_mv, 'arima_mv':arima_mv.arima_mv, 
                        'xgboost_mv':xgboost_mv.xgboost_mv, 'prophet_mv':prophet_mv.prophet_mv, 'olsregression_mv':olsregression_mv.olsregression_mv, 
                        'random_forest_mv' : random_forest_mv.random_forest_mv}
                        #TODO 'light_gbm_mv':light_gbm_mv.lgbm_mv, 'causalimpact_mv':causal_impact_mv.causalimpact_mv
                    

list_of_models = univariate_models.copy()
if multivariate_flag == 1:  list_of_models.update(multivariate_models)


In [145]:
df_output = data[data['index']=='U00000000006032021'].apply(lambda x: forecast_engine_pipeline.forecast_engine(x, months_to_forecast,debug_flag,debug_models,list_of_models, single_series,  all_at_once_flag, validation_window, max_validation, outlier_flag, select_variable_flag),axis = 1)    

U00000000006032021
       Header     Value  Avg Temp (C) Inflation Data Lending Rate  \
38  2022 - 05   58810.0     22.117168         0.0765         0.07   
39  2022 - 06   51045.0     20.450507         0.0799       0.0775   
40  2022 - 07   89554.0     20.463593         0.0815       0.0775   
41  2022 - 08  121024.0     20.463593          0.087        0.085   
42  2022 - 09   30414.0     20.463593          0.087       0.0925   
43  2022 - 10  155549.0     20.463593         0.0841       0.0925   
44  2022 - 11  124784.0     20.463593          0.078          0.1   
45  2022 - 12   90784.0     20.463593         0.0782        0.105   
46  2023 - 01       NaN           NaN            NaN          NaN   
47  2023 - 02       NaN           NaN            NaN          NaN   
48  2023 - 03       NaN           NaN            NaN          NaN   
49  2023 - 04       NaN           NaN            NaN          NaN   
50  2023 - 05       NaN           NaN            NaN          NaN   
51  2023 - 06  

/Users/anand_k/Downloads/UPL/Forecast Engine/forecast_engine/modelling/models/ml_models/holt_winters.py:59: FutureWarning: the 'damped'' keyword is deprecated, use 'damped_trend' instead.
  fit = ets(cv_train.values,trend=self.trend,damped=self.damped,seasonal=self.seasonality,seasonal_periods=seasonal_periods).fit(optimized=True, start_params=None)
/Users/anand_k/Downloads/UPL/Forecast Engine/forecast_engine/modelling/models/ml_models/holt_winters.py:59: FutureWarning: the 'damped'' keyword is deprecated, use 'damped_trend' instead.
  fit = ets(cv_train.values,trend=self.trend,damped=self.damped,seasonal=self.seasonality,seasonal_periods=seasonal_periods).fit(optimized=True, start_params=None)
/Users/anand_k/Downloads/UPL/Forecast Engine/forecast_engine/modelling/models/ml_models/holt_winters.py:59: FutureWarning: the 'damped'' keyword is deprecated, use 'damped_trend' instead.
  fit = ets(cv_train.values,trend=self.trend,damped=self.damped,seasonal=self.seasonality,seasonal_periods=s

Optimization not converged
Optimization not converged
Optimization not converged
Optimization not converged
Optimization not converged
Optimization not converged
hw_ets_r_MNA : errored_out
hw_ets_r_MAA : errored_out


/Users/anand_k/Downloads/UPL/Forecast Engine/forecast_engine/modelling/models/ml_models/linear_regression_mv.py:104: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  feature_scores = {'linearregression_mv':(pd.Series(index=regressors.columns).sort_values(ascending=False))}
/Users/anand_k/Downloads/UPL/Forecast Engine/forecast_engine/modelling/models/ml_models/random_forest_mv.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  select_oos.replace(0, np.nan, inplace=True)
Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.
/Users/anand_k/Downloads/UPL/Forecast Engine/venv/lib/python3.9/site-packages/pmdarima/arima/_validation.py

light_gbm : errored_out
rw_drift : errored_out


...

/Users/anand_k/Downloads/UPL/Forecast Engine/venv/lib/python3.9/site-packages/fbprophet/forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/anand_k/Downloads/UPL/Forecast Engine/venv/lib/python3.9/site-packages/fbprophet/forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/anand_k/Downloads/UPL/Forecast Engine/forecast_engine/modelling/models/ml_models/prophet_mv.py:105: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  feature_scores = {'prophet_mv':(pd.Series(index=ti.iloc[:,1:].columns).sort_values(ascending=False))}


causalimpact : errored_out


/Users/anand_k/Downloads/UPL/Forecast Engine/venv/lib/python3.9/site-packages/fbprophet/forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/anand_k/Downloads/UPL/Forecast Engine/venv/lib/python3.9/site-packages/fbprophet/forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/anand_k/Downloads/UPL/Forecast Engine/venv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
250 fits failed out of a total of 1250.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
---------------------------------

492.30857586860657
                 models      smape
0                 arima  18.413012
1          hw_ets_r_MNN  22.030438
2            hw_ets_AFN  23.935076
3            hw_ets_ATN  24.498363
4      random_forest_mv  25.103374
5               croston  25.213265
6                 naive  25.213266
7            hw_ets_NFN  27.095971
8             naive_ets  27.099489
9            prophet_mv  27.108402
10         hw_ets_r_MAN  29.106201
11           xgboost_mv  30.610703
12              prophet  31.647479
13           hw_ets_ATA  31.979755
14           hw_ets_NFA  32.041404
15           hw_ets_AFA  32.107478
16             arima_mv  34.496275
17               snaive  36.293406
18           mean_model  36.530253
19  linearregression_mv  37.955300
20                garch  43.335459
21    snaive_twoseasons  49.959520
22         median_model  64.903240
oos_forecast before:
        date_      forecast method  Actual
0  2023 - 01  58653.757012  arima     NaN
1  2023 - 02  98768.121990  arima  

<Figure size 640x480 with 0 Axes>

In [146]:
df_output = df_output.ravel()
output_temp = df_output[0]['output']
df_classifier_train_set = df_output[0]['classifier_train_set']
impact_temp = df_output[0]['var_coeff']
df_univariate_temp = df_output[0]['df_univariate']
mp = df_output[0]['df_multivariate']
df_ensemble_temp = df_output[0]['df_ensemble']
df_fitted_values_list = df_output[0]['fitted_values_list']
model_list = df_output[0]['dataf']

In [160]:
model_list

,models,smape
0,arima,18.413012
1,hw_ets_r_MNN,22.030438
2,hw_ets_AFN,23.935076
3,hw_ets_ATN,24.498363
4,random_forest_mv,25.103374
5,croston,25.213265
6,naive,25.213266
7,hw_ets_NFN,27.095971
8,naive_ets,27.099489
9,prophet_mv,27.108402


In [148]:
output_temp

,index,DATE,Forecast,Method,Forecast_accuracy,Actual,timing,pre_timing,mid_timing,post_timing,total
0,U00000000006032021,2022 - 10,78092.060523,arima,18.413012,155549.0,492.648564,0.014858,492.652792,0.005865,492.673515
1,U00000000006032021,2022 - 11,103255.789812,arima,18.413012,124784.0,492.648564,0.014858,492.652792,0.005865,492.673515
2,U00000000006032021,2022 - 12,101137.172767,arima,18.413012,90784.0,492.648564,0.014858,492.652792,0.005865,492.673515
3,U00000000006032021,2023 - 01,58653.757012,arima,NaN,NaN,492.648564,0.014858,492.652792,0.005865,492.673515
4,U00000000006032021,2023 - 02,98768.121990,arima,NaN,NaN,492.648564,0.014858,492.652792,0.005865,492.673515
5,U00000000006032021,2023 - 03,93697.013200,arima,NaN,NaN,492.648564,0.014858,492.652792,0.005865,492.673515


In [149]:
df_ensemble_temp

,index,0,date_,forecast,method
0,U00000000006032021,0,2022 - 09,75352.854675,arima hw_ets_r_MNN hw_ets_AFN hw_ets_ATN rando...
1,U00000000006032021,0,2022 - 10,78442.441995,arima hw_ets_r_MNN hw_ets_AFN hw_ets_ATN rando...
2,U00000000006032021,0,2022 - 11,87825.535017,arima hw_ets_r_MNN hw_ets_AFN hw_ets_ATN rando...
3,U00000000006032021,0,2022 - 12,91529.441902,arima hw_ets_r_MNN hw_ets_AFN hw_ets_ATN rando...
4,U00000000006032021,0,2023 - 01,76111.594810,arima hw_ets_r_MNN hw_ets_AFN hw_ets_ATN rando...
5,U00000000006032021,0,2023 - 02,91868.794275,arima hw_ets_r_MNN hw_ets_AFN hw_ets_ATN rando...
6,U00000000006032021,0,2023 - 03,88396.817277,arima hw_ets_r_MNN hw_ets_AFN hw_ets_ATN rando...


In [150]:
df_classifier_train_set

,index,hw_ets_r_MNN,hw_ets_ATA,hw_ets_AFA,hw_ets_ATN,hw_ets_NFA,hw_ets_NFN,hw_ets_AFN,hw_ets_r_MAA,hw_ets_r_MAN,...,linearregression_mv,xgboost_mv,prophet,causalimpact,light_gbm,random_forest_mv,arima_mv,prophet_mv,olsregression_mv,date_
0,U00000000006032021,1,0,0,1,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,2023 - 01


DEBUG THE ENGINE

In [136]:
# Function test
ti = data[data['index']=='U00000000006032021'].squeeze(axis=0)
from pandas import MultiIndex
import datetime 
single_series = 0
ts1 = datetime.datetime.now().timestamp()
if (single_series == 0):
  ti = pd.Series(ti)
series_key = str(ti['index'])
print(series_key)
ti1, ti_key, variable_list, series_flag = utils.get_ti_in_shape(ti.copy())
ti = ti1.copy()
ts2 = datetime.datetime.now().timestamp()
ti1 = utils.driver_imputation(ti.copy())
ti=ti1.copy()
ti = ti[2:]
  
forecast_loop.forecast_loop_function(ti, series_flag, months_to_forecast, debug_flag, debug_models, list_of_models, variable_list, single_series, all_at_once_flag, validation_window, max_validation, outlier_flag, select_variable_flag)

U00000000006032021
       Header     Value  Avg Temp (C) Inflation Data Lending Rate  \
38  2022 - 05   58810.0     22.117168         0.0765         0.07   
39  2022 - 06   51045.0     20.450507         0.0799       0.0775   
40  2022 - 07   89554.0     20.463593         0.0815       0.0775   
41  2022 - 08  121024.0     20.463593          0.087        0.085   
42  2022 - 09   30414.0     20.463593          0.087       0.0925   
43  2022 - 10  155549.0     20.463593         0.0841       0.0925   
44  2022 - 11  124784.0     20.463593          0.078          0.1   
45  2022 - 12   90784.0     20.463593         0.0782        0.105   
46  2023 - 01       NaN           NaN            NaN          NaN   
47  2023 - 02       NaN           NaN            NaN          NaN   
48  2023 - 03       NaN           NaN            NaN          NaN   
49  2023 - 04       NaN           NaN            NaN          NaN   
50  2023 - 05       NaN           NaN            NaN          NaN   
51  2023 - 06  

/Users/anand_k/Downloads/UPL/Forecast Engine/venv/lib/python3.9/site-packages/statsmodels/tsa/holtwinters/model.py:915: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/Users/anand_k/Downloads/UPL/Forecast Engine/venv/lib/python3.9/site-packages/statsmodels/tsa/holtwinters/model.py:915: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/Users/anand_k/Downloads/UPL/Forecast Engine/venv/lib/python3.9/site-packages/statsmodels/tsa/holtwinters/model.py:915: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/Users/anand_k/Downloads/UPL/Forecast Engine/venv/lib/python3.9/site-packages/statsmodels/tsa/holtwinters/model.py:915: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/Users/anand_k/Downloads/UPL/Forecast Engine/venv/lib/python3.9/site-packages/statsmodels/tsa/holtwinters/model.py:915: ConvergenceWarning: Optimization failed to c

naive_ets : errored_out
snaive_twoseasons : errored_out
rw_drift : errored_out
croston : errored_out
arima : errored_out
garch : errored_out
linearregression_mv : errored_out


Importing plotly failed. Interactive plots will not work.


xgboost_mv : errored_out

Initial log joint probability = 10.4864
Iteration  1. Log joint probability =    10.6763. Improved by 0.189869.
Iteration  2. Log joint probability =     23.856. Improved by 13.1796.
Iteration  3. Log joint probability =    32.1902. Improved by 8.33421.
Iteration  4. Log joint probability =    32.9769. Improved by 0.786728.
Iteration  5. Log joint probability =     33.274. Improved by 0.297127.
Iteration  6. Log joint probability =      33.38. Improved by 0.105934.
Iteration  7. Log joint probability =    33.3862. Improved by 0.00621114.
Iteration  8. Log joint probability =    33.3906. Improved by 0.00446816.
Iteration  9. Log joint probability =       33.4. Improved by 0.0093533.
Iteration 10. Log joint probability =    33.4057. Improved by 0.00567456.
Iteration 11. Log joint probability =    33.4493. Improved by 0.0436376.
Iteration 12. Log joint probability =    33.4656. Improved by 0.0162629.
Iteration 13. Log joint probability =    33.4765. Improved by 0

{'dataf':          models      smape
 0         naive   9.429957
 1  median_model  38.593451
 2       prophet  41.298547
 3        snaive  50.368919
 4    prophet_mv  54.196962,
 'forecast_df':        date_  forecast   type  Forecast_accuracy    Actual     timing
 0  2022 - 10  121024.0  naive           9.429957  155549.0  31.337568
 1  2022 - 11  121024.0  naive           9.429957  124784.0  31.337568
 2  2022 - 12  121024.0  naive           9.429957   90784.0  31.337568
 3  2023 - 01   90784.0  naive                NaN       NaN  31.337568
 4  2023 - 02   90784.0  naive                NaN       NaN  31.337568
 5  2023 - 03   90784.0  naive                NaN       NaN  31.337568,
 'classifier_train_set': 0  hw_ets_r_MNN  hw_ets_ATA  hw_ets_AFA  hw_ets_ATN  hw_ets_NFA  hw_ets_NFN  \
 0             0           0           0           0           0           0   
 
 0  hw_ets_AFN  hw_ets_r_MAA  hw_ets_r_MAN  hw_ets_r_MNA  ...  \
 0           0             0             0             0  

<Figure size 640x480 with 0 Axes>